In [19]:
import clip
import torch
import pandas as pd
import numpy as np
from PIL import Image
import glob

In [20]:
%matplotlib inline

print('\nLoading model...')
available_models = ['RN50', 'RN101', 'RN50x4', 'RN50x16']
layers = ['layer4', 'layer3', 'layer2', 'layer1']

clip_model = available_models[0]
saliency_layer = layers[0]

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load(clip_model, device=device, jit=False)
print(f"Done! Model loaded to {device} device")


Loading model...
Done! Model loaded to cuda device


In [21]:
path = "/home/lazye/Documents/ufrgs/mcs/datasets/FairFace/"
fface_df = pd.read_csv(f"{path}/train/fairface_label_train.csv")

In [22]:
fface_df.head()

,file,age,gender,race,service_test
0,train/1.jpg,50-59,Male,East Asian,True
1,train/2.jpg,30-39,Female,Indian,False
2,train/3.jpg,3-9,Female,Black,False
3,train/4.jpg,20-29,Female,Indian,True
4,train/5.jpg,20-29,Female,Indian,True


In [23]:
woman_embds_df = pd.read_pickle('woman_embeddings.csv')
man_embds_df = pd.read_pickle('man_embeddings.csv')

In [24]:
def get_scores(img_embd, classes):
    """Softmax of pairs of 'img_embd' and each class of 'classes'"""
    image_features = torch.from_numpy(img_embd).to(device)

    text_inputs = torch.cat(
        [clip.tokenize(f"a photo of a {c}") for c in classes]).to(device)
    
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
    
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)

    values, indices = similarity[0].topk(len(similarity[0]))
    scores = list()
    for value, index in zip(values, indices):
        scores.append((classes[index], round(100 * value.item(), 2)))
    return scores


In [25]:
labels = ['man', 'woman']
print(get_scores(man_embds_df.iloc[0]['embeddings'], labels))

[('man', 82.23), ('woman', 17.79)]


In [26]:
def eval_embds(emb_df, classes):
    results = dict()

    text_inputs = torch.cat(
        [clip.tokenize(f"a photo of a {c}") for c in classes]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    for _, emb in emb_df.iterrows():
        name = emb['file']
        image_features = emb['embeddings']
        image_features = torch.from_numpy(image_features).to(device)
        similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)

        values, indices = similarity[0].topk(len(similarity[0]))
        scores = list()
        for value, index in zip(values, indices):
            scores.append((classes[index], round(100 * value.item(), 2)))
        results[name] = scores
    
    return results

In [27]:
man_results = eval_embds(man_embds_df, labels)

In [28]:
woman_results = eval_embds(woman_embds_df, labels)

In [29]:
gender_scores = dict()
files = list()
predicts = list()

for key, value in man_results.items():
    files.append(key)
    win_class = value[0][0]
    if win_class == "woman":
        predicts.append('Female')
    else:
        predicts.append('Male')

for key, value in woman_results.items():
    files.append(key)
    win_class = value[0][0]
    if win_class == "woman":
        predicts.append('Female')
    else:
        predicts.append('Male')

In [30]:
gender_scores['file'] = files
gender_scores['gender_preds'] = predicts

In [31]:
gender_scores = pd.DataFrame(data=gender_scores)

In [32]:
fface_df = fface_df.set_index('file').join(gender_scores.set_index('file'))

In [33]:
fface_df[fface_df['gender'] != fface_df['gender_preds']]

,age,gender,race,service_test,gender_preds
file,,,,,
train/6.jpg,20-29,Male,White,True,Female
train/8.jpg,30-39,Female,Indian,True,Male
train/16.jpg,30-39,Female,White,False,Male
train/19.jpg,0-2,Female,Black,False,Male
train/30.jpg,20-29,Female,Black,False,Male
...,...,...,...,...,...
train/86711.jpg,3-9,Male,Black,False,Female
train/86722.jpg,0-2,Male,Black,False,Female
train/86723.jpg,20-29,Female,Middle Eastern,True,Male


In [34]:
fface_df.to_csv('./fface_gender_preds.csv')